In [1]:
!pip -q install -U ultralytics==8.3.50 kagglehub

from ultralytics import YOLO
import kagglehub, yaml, os, glob, torch

# dataset
ds_path = kagglehub.dataset_download("gabrielfcarvalho/cardd-with-yolo-annotations-images-labels")
data = {
    "train": os.path.join(ds_path, "train/images"),
    "val":   os.path.join(ds_path, "val/images"),
    "test":  os.path.join(ds_path, "test/images"),
    "nc": 6,
    "names": ['dent','scratch','crack','glass shatter','lamp broken','tire flat']
}
fixed_yaml = "/content/data_fixed.yaml"
with open(fixed_yaml, "w") as f: yaml.safe_dump(data, f, sort_keys=False)

print("torch:", torch.__version__, "cuda:", torch.cuda.is_available())
print(open(fixed_yaml).read())


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.0/899.0 kB 41.5 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 2.80G/2.80G [02:25<00:00, 20.6MB/s]

Extracting files...


torch: 2.8.0+cu126 cuda: True
train: /root/.cache/kagglehub/datasets/gabrielfcarvalho/cardd-with-yolo-annotations-images-labels/versions/2/train/images
val: /root/.cache/kagglehub/datasets/gabrielfcarvalho/cardd-with-yolo-annotations-images-labels/versions/2/val/images
test: /root/.cache/kagglehub/datasets/gabrielfcarvalho/cardd-with-yolo-annotations-images-labels/versions/2/test/images
nc: 6
names:
- dent
- scratch
- crack
- glass shatter
- lamp broken
- tire flat



In [2]:
from google.colab import files
up = files.upload()  # choose your best.pt; it will appear under /content
# Suppose the uploaded file is named exactly 'best.pt'
weights_path = "/content/best.pt"


Saving best.pt to best.pt


In [ ]:
model = YOLO(weights_path)  # loads your best.pt

results = model.train(
    data=fixed_yaml,
    epochs=7,              # do 7 more; you can set 10 if you want a bit of smoothing
    imgsz=1024,            # keep same imgsz you used for S-model
    batch=16,              # adjust if you hit OOM
    device=0,
    workers=2,
    amp=True,

    # small-object friendly augs (same as before, maybe slightly milder)
    hsv_h=0.015, hsv_s=0.6, hsv_v=0.35,
    translate=0.06, scale=0.45, perspective=0.0005,
    mosaic=0.15, mixup=0.05, copy_paste=0.03,
    fliplr=0.5, flipud=0.0,

    # IMPORTANT: since we're topping off from best.pt, use a smaller LR and little/no warmup
    lr0=0.005,     # half the previous 0.01
    lrf=0.1,
    warmup_epochs=0.5,  # or 0
    patience=5,         # short patience is fine for a short run

    project="/content/runs_yolo11s",
    name="finetune_from_best",
    save=True, save_period=0
)

run_dir = results.save_dir
print("Run dir:", run_dir)


New https://pypi.org/project/ultralytics/8.3.199 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.50 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/content/best.pt, data=/content/data_fixed.yaml, epochs=7, time=None, patience=5, batch=16, imgsz=1024, save=True, save_period=0, cache=False, device=0, workers=2, project=/content/runs_yolo11s, name=finetune_from_best, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=N

100%|██████████| 755k/755k [00:00<00:00, 21.4MB/s]



                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  3                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  4                  -1  1    103360  ultralytics.nn.modules.block.C3k2            [128, 256, 1, False, 0.25]    
  5                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  6                  -1  1    346112  ultralytics.nn.modules.block.C3k2            [256, 256, 1, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256

100%|██████████| 5.35M/5.35M [00:00<00:00, 106MB/s]


AMP: checks passed ✅


...

        7/7        11G      1.157      1.075      1.543         31       1024: 100%|██████████| 176/176 [02:38<00:00,  1.11it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:18<00:00,  1.39it/s]

                   all        810       1744      0.749      0.672      0.709      0.507



7 epochs completed in 0.364 hours.
Optimizer stripped from /content/runs_yolo11s/finetune_from_best/weights/last.pt, 19.2MB
Optimizer stripped from /content/runs_yolo11s/finetune_from_best/weights/best.pt, 19.2MB

Validating /content/runs_yolo11s/finetune_from_best/weights/best.pt...
Ultralytics 8.3.50 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 238 layers, 9,415,122 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:19<00:00,  1.32it/s]


                   all        810       1744      0.751      0.671      0.709      0.507
                  dent        352        501      0.663      0.435      0.522      0.264
               scratch        431        728      0.629      0.468      0.505      0.266
                 crack        122        177      0.504      0.458      0.423      0.197
         glass shatter        134        135      0.976      0.985      0.994      0.842
           lamp broken        139        141      0.765      0.794      0.861      0.681
             tire flat         59         62      0.967      0.887       0.95      0.794
Speed: 0.4ms preprocess, 8.5ms inference, 0.0ms loss, 3.3ms postprocess per image
Results saved to /content/runs_yolo11s/finetune_from_best
Run dir: /content/runs_yolo11s/finetune_from_best


In [ ]:
best_new = YOLO(os.path.join(run_dir, "weights", "best.pt"))
val_metrics  = best_new.val(data=fixed_yaml, split="val",  imgsz=1024)
test_metrics = best_new.val(data=fixed_yaml, split="test", imgsz=1024)
print("VAL:", val_metrics)
print("TEST:", test_metrics)

# quick preview
test_dir = os.path.join(ds_path, "test/images")
samples = sorted(glob.glob(os.path.join(test_dir, "*.jpg")))[:12] or \
          sorted(glob.glob(os.path.join(test_dir, "*.png")))[:12]
best_new.predict(source=samples if samples else test_dir,
                 imgsz=1024, conf=0.25, iou=0.6,
                 save=True, project="/content/preds_vis", name="y11s_best")

# download the new weights
from google.colab import files
files.download(os.path.join(run_dir, "weights", "best.pt"))
files.download(os.path.join(run_dir, "weights", "last.pt"))


Ultralytics 8.3.50 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 238 layers, 9,415,122 parameters, 0 gradients, 21.3 GFLOPs


val: Scanning /root/.cache/kagglehub/datasets/gabrielfcarvalho/cardd-with-yolo-annotations-images-labels/versions/2/val/labels.cache... 810 images, 0 backgrounds, 0 corrupt: 100%|██████████| 810/810 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 51/51 [00:21<00:00,  2.36it/s]


                   all        810       1744      0.751      0.672      0.708      0.507
                  dent        352        501      0.668      0.435      0.522      0.263
               scratch        431        728      0.625      0.466      0.498      0.265
                 crack        122        177      0.506      0.458       0.42      0.195
         glass shatter        134        135      0.969      0.985      0.993      0.845
           lamp broken        139        141      0.769      0.803      0.864      0.684
             tire flat         59         62      0.967      0.887       0.95      0.793
Speed: 0.6ms preprocess, 17.7ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/val
Ultralytics 8.3.50 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)


val: Scanning /root/.cache/kagglehub/datasets/gabrielfcarvalho/cardd-with-yolo-annotations-images-labels/versions/2/test/labels... 374 images, 0 backgrounds, 0 corrupt: 100%|██████████| 374/374 [00:00<00:00, 408.26it/s]

val: New cache created: /root/.cache/kagglehub/datasets/gabrielfcarvalho/cardd-with-yolo-annotations-images-labels/versions/2/test/labels.cache


...

                   all        374        785      0.719      0.671      0.713      0.516
                  dent        157        236      0.602      0.538      0.554      0.297
               scratch        183        307      0.663      0.507      0.585      0.307
                 crack         48         70      0.422      0.365      0.389      0.222
         glass shatter         71         71      0.909      0.972       0.98      0.818
           lamp broken         65         69      0.853      0.768       0.86      0.667
             tire flat         31         32      0.868      0.875      0.909      0.783
Speed: 0.5ms preprocess, 17.3ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to runs/detect/val2
VAL: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a5f608f30b0>
curves: ['Precision

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip -q install -U ultralytics==8.3.50 kagglehub
from ultralytics import YOLO
import kagglehub, yaml, os, pandas as pd, numpy as np

# (A) Recreate dataset yaml (if not already)
ds_path = kagglehub.dataset_download("gabrielfcarvalho/cardd-with-yolo-annotations-images-labels")
data = {
    "train": os.path.join(ds_path, "train/images"),
    "val":   os.path.join(ds_path, "val/images"),
    "test":  os.path.join(ds_path, "test/images"),
    "nc": 6,
    "names": ['dent','scratch','crack','glass shatter','lamp broken','tire flat']
}
fixed_yaml = "/content/data_fixed.yaml"
with open(fixed_yaml, "w") as f: yaml.safe_dump(data, f, sort_keys=False)

# (B) Load your weights (uploaded earlier via files.upload)
weights_path = "/content/best.pt"  # change if your filename differs
model = YOLO(weights_path)

# (C) Validate on test (or 'val') and request plots
metrics = model.val(
    data=fixed_yaml,
    split="test",          # or "val" if you prefer
    imgsz=1024,            # match your training setting
    conf=0.001,            # low conf for fair mAP calc
    iou=0.6,
    plots=True,            # saves PR/F1/confusion plots
    save_json=True         # saves COCO JSON for auditing
)

# ---- Overall metrics (robust extraction across versions) ----
def get(d, key, default=None):
    return d.get(key, default) if isinstance(d, dict) else default

rd = getattr(metrics, "results_dict", {})  # preferred
overall = {
    "precision": get(rd, "metrics/precision(B)", None),
    "recall":    get(rd, "metrics/recall(B)", None),
    "mAP@.5":    get(rd, "metrics/mAP50(B)", None),
    "mAP@.5:.95":get(rd, "metrics/mAP50-95(B)", None),
}

# Fallbacks if results_dict isn’t populated (older/newer API changes)
if overall["mAP@.5:.95"] is None:
    # try common attributes
    overall["mAP@.5:.95"] = getattr(getattr(metrics, "box", metrics), "map", None)
if overall["mAP@.5"] is None:
    overall["mAP@.5"] = getattr(getattr(metrics, "box", metrics), "map50", None)
if overall["precision"] is None:
    overall["precision"] = getattr(metrics, "precision", None)
if overall["recall"] is None:
    overall["recall"] = getattr(metrics, "recall", None)

print("Overall metrics:", overall)

# ---- Per-class AP table ----
# Ultralytics typically stores per-class AP@.5:.95 in metrics.box.maps
names = model.names if hasattr(model, "names") else {i:n for i,n in enumerate(data["names"])}
maps = None
try:
    maps = metrics.box.maps  # AP@.5:.95 per class
except Exception:
    maps = getattr(metrics, "maps", None)

# Best-effort AP@.5 per class (if available)
ap50s = None
try:
    ap50s = metrics.box.map50s  # some versions expose this
except Exception:
    ap50s = None  # may not exist; OK to leave None

ncls = len(names)
df = pd.DataFrame({
    "class_id": list(range(ncls)),
    "class_name": [names[i] for i in range(ncls)],
    "AP@.5:.95": maps if maps is not None else [np.nan]*ncls,
    "AP@.5":     ap50s if ap50s is not None else [np.nan]*ncls
})
df.sort_values("AP@.5:.95", ascending=False, inplace=True)
df.reset_index(drop=True, inplace=True)
df


Using Colab cache for faster access to the 'cardd-with-yolo-annotations-images-labels' dataset.
Ultralytics 8.3.50 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 238 layers, 9,415,122 parameters, 0 gradients, 21.3 GFLOPs


val: Scanning /kaggle/input/cardd-with-yolo-annotations-images-labels/test/labels... 374 images, 0 backgrounds, 0 corrupt: 100%|██████████| 374/374 [00:03<00:00, 104.09it/s]


val: WARNING ⚠️ Cache directory /kaggle/input/cardd-with-yolo-annotations-images-labels/test is not writeable, cache not saved.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:14<00:00,  1.64it/s]


                   all        374        785      0.811      0.679      0.739      0.547
                  dent        157        236      0.682      0.563      0.589      0.314
               scratch        183        307      0.719      0.573      0.642      0.349
                 crack         48         70      0.709      0.414      0.462       0.27
         glass shatter         71         71      0.898      0.972       0.97      0.826
           lamp broken         65         69      0.923      0.768      0.878      0.706
             tire flat         31         32      0.935      0.781      0.897      0.816
Speed: 0.5ms preprocess, 17.0ms inference, 0.0ms loss, 3.3ms postprocess per image
Saving runs/detect/val3/predictions.json...
Results saved to runs/detect/val3
Overall metrics: {'precision': np.float64(0.8109429385445489), 'recall': np.float64(0.6785548139321665), 'mAP@.5': np.float64(0.7394953311800295), 'mAP@.5:.95': np.float64(0.5468216804012601)}


,class_id,class_name,AP@.5:.95,AP@.5
0,3,glass shatter,0.826105,NaN
1,5,tire flat,0.815937,NaN
2,4,lamp broken,0.706107,NaN
3,1,scratch,0.348548,NaN
4,0,dent,0.313855,NaN
5,2,crack,0.270377,NaN


Next part

In [3]:
from ultralytics import YOLO
import shutil

# Load your trained model
model = YOLO("/content/best.pt")

# Validate on val set
metrics_val = model.val(
    data="/content/data_fixed.yaml",
    split="val",
    imgsz=1024,
    conf=0.001,
    iou=0.6,
    plots=True,         # generate PR, F1, confusion, etc.
    save_json=True      # saves COCO-style results.json
)

print("Validation artifacts saved in:", metrics_val.save_dir)


Ultralytics 8.3.50 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 238 layers, 9,415,122 parameters, 0 gradients, 21.3 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 94.4MB/s]
val: Scanning /root/.cache/kagglehub/datasets/gabrielfcarvalho/cardd-with-yolo-annotations-images-labels/versions/2/val/labels... 810 images, 0 backgrounds, 0 corrupt: 100%|██████████| 810/810 [00:00<00:00, 2221.09it/s]

val: New cache created: /root/.cache/kagglehub/datasets/gabrielfcarvalho/cardd-with-yolo-annotations-images-labels/versions/2/val/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 51/51 [00:22<00:00,  2.31it/s]


                   all        810       1744       0.75      0.681      0.716      0.507
                  dent        352        501      0.669      0.465      0.529      0.261
               scratch        431        728      0.634      0.475      0.511      0.266
                 crack        122        177      0.509      0.463      0.437      0.195
         glass shatter        134        135      0.967      0.985      0.994      0.846
           lamp broken        139        141      0.779      0.809      0.871      0.683
             tire flat         59         62      0.944      0.887      0.952      0.793
Speed: 1.5ms preprocess, 15.2ms inference, 0.0ms loss, 2.1ms postprocess per image
Saving runs/detect/val/predictions.json...
Results saved to runs/detect/val
Validation artifacts saved in: runs/detect/val


In [4]:
import pandas as pd

rd = getattr(metrics_val, "results_dict", {})
overall = {
    "Precision": rd.get("metrics/precision(B)", None),
    "Recall": rd.get("metrics/recall(B)", None),
    "mAP@0.5": rd.get("metrics/mAP50(B)", None),
    "mAP@0.5:0.95": rd.get("metrics/mAP50-95(B)", None),
}
print(overall)

# save as CSV for presentation
pd.DataFrame([overall]).to_csv("/content/overall_metrics.csv", index=False)


{'Precision': np.float64(0.7502720322153085), 'Recall': np.float64(0.6807356698981607), 'mAP@0.5': np.float64(0.7157379914341359), 'mAP@0.5:0.95': np.float64(0.507423704326548)}


In [5]:
names = model.names
maps = metrics_val.box.maps  # AP@.5:.95 per class
df = pd.DataFrame({
    "Class": [names[i] for i in range(len(names))],
    "AP@0.5:0.95": maps
})
df.to_csv("/content/per_class_metrics.csv", index=False)
df


,Class,AP@0.5:0.95
0,dent,0.261068
1,scratch,0.266212
2,crack,0.194732
3,glass shatter,0.846359
4,lamp broken,0.683479
5,tire flat,0.792692


In [6]:
# zip the entire validation results folder
shutil.make_archive("/content/val_results", "zip", metrics_val.save_dir)

from google.colab import files
files.download("/content/val_results.zip")   # contains graphs & plots
files.download("/content/overall_metrics.csv")
files.download("/content/per_class_metrics.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
df=pd.read_csv('overall_metrics.csv')
df

,Precision,Recall,mAP@0.5,mAP@0.5:0.95
0,0.750272,0.680736,0.715738,0.507424
